In [1]:
import chess
import pandas as pd
from extract_features import board_features


In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
last_board = chess.Board("r2qkb1r/ppp2ppp/2B2n2/3pp3/3P1B2/2N1Pb2/PPP2PPP/R2QK2R b KQkq - 0 7")
current_board = chess.Board("3rkb1r/1B1bqpp1/p7/3QN2p/4PP2/1PN5/P1P2P1P/2KR3R b k - 0 17")

In [4]:
last_board_features = board_features(last_board)
current_board_features = board_features(current_board)

In [5]:
current_board_features.sort_index(inplace=True,axis=1)

In [6]:
last_board_features.sort_index(inplace=True,axis=1)

In [7]:
last_board_features

,B_white,Check_black,Check_white,Doubled_pawns_black,Doubled_pawns_white,Isolated_pawns_black,Isolated_pawns_white,K_white,King_pawn_shield_black,King_pawn_shield_white,King_zone_attacked_squares_black,King_zone_attacked_squares_white,King_zone_controlled_squares_black,King_zone_controlled_squares_white,Mobility_black,Mobility_white,N_white,P_white,Passed_pawns_black,Passed_pawns_white,Q_white,R_white,Result,Trapped_bishops_black,Trapped_bishops_white,Trapped_queens_black,Trapped_queens_white,Trapped_rooks_black,Trapped_rooks_white,b_black,k_black,n_black,p_black,q_black,r_black
0,2,1,0,0,0,0,0,1,1,2,2,2,5,6,4,40,1,8,0,0,1,2,0,1,0,0,0,2,2,2,1,1,8,1,2


In [8]:
import pandas as pd
import numpy as np
#import openpyxl
import os
from tqdm import tqdm
tqdm.pandas()

import tensorflow as tf


from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import tensorflow_decision_forests as tfdf

2024-03-21 09:49:21.672078: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 09:49:21.735564: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 09:49:21.735603: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 09:49:21.736928: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-21 09:49:21.745687: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 09:49:21.747039: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [9]:
model = tf.keras.models.load_model("../data/6_models/Elo_1500_1600.csv.pkl")

2024-03-21 09:49:24.564541: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 09:49:24.565074: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
[INFO 24-03-21 09:49:24.8565 WET kernel.cc:1233] Loading model from path ../data/6_models/Elo_1500_1600.csv.pkl/assets/ with prefix d42114d41b1b4946
[INFO 24-03-21 09:49:25.3986 WET decision_forest.cc:660] Model loaded with 24 root(s), 302976 node(s), and 32 input feature(s).
[INFO 24-03-21 09:49:25.3987 WET abstract_model.cc:1344] Engine "RandomForestOptPred" built
[INF

In [10]:
importance = model.make_inspector().variable_importances()

AttributeError: 'RandomForestModel' object has no attribute 'make_inspector'

In [29]:
last_board_features_ds = tfdf.keras.pd_dataframe_to_tf_dataset(last_board_features, label="Result")


In [30]:
model.predict(last_board_features_ds)

1/1 [==============================] - 0s 222ms/step


array([[0.29333314]], dtype=float32)

In [31]:
last_board_features.compare(current_board_features)

B_white       Check_black       Doubled_pawns_white        \
     self other        self other                self other   
0       2     1           1     0                   0     1   

  Isolated_pawns_black       Isolated_pawns_white        \
                  self other                 self other   
0                    0     2                    0     2   

  King_zone_attacked_squares_white       King_zone_controlled_squares_black  \
                              self other                               self   
0                                2     0                                  5   

        King_zone_controlled_squares_white       Mobility_black        \
  other                               self other           self other   
0     6                                  6     5              4    29   

  Mobility_white       N_white       P_white       Passed_pawns_white        \
            self other    self other    self other               self other   
0             40    47       1     2       8     7                  0     1   

  Trapped_rooks_black       Trapped_rooks_white       n_black       p_black  \
                 self other                self other    self other    self   
0                   2     1                   2     1       1     0       8   

         
  other  
0     4

In [17]:
current_board_features.subtract(last_board_features,fill_value=0)

,Doubled_pawns_black,Doubled_pawns_white,Isolated_pawns_black,Isolated_pawns_white,King_pawn_shield_black,King_pawn_shield_white,King_zone_attacked_squares_black,King_zone_attacked_squares_white,King_zone_controlled_squares_black,King_zone_controlled_squares_white,Mobility_black,Mobility_white,Passed_pawns_black,Passed_pawns_white,Result
0,0,1,0,1,0,0,0,0,0,0,-7,-1,0,0,0
